### General Considerations

The prompt will be structured as follows: 

System: < Describes role >. < Describes task >. < Defines output contract >. < Defines constraints >. < Examples >.

User: < PR input >

In [1]:
import requests
import json
import pprint
import re

In [2]:
def prompt_formatting(system, user):
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user}
    ]

In [3]:
def extract_json_from_response(response):
    # Find JSON objects - simple approach without recursive regex
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    
    match = re.search(json_pattern, response, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            # Try to find JSON arrays
            array_pattern = r'\[\s*\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}\s*(?:,\s*\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}\s*)*\]'
            array_match = re.search(array_pattern, response, re.DOTALL)
            if array_match:
                array_str = array_match.group(0)
                try:
                    return json.loads(array_str)
                except json.JSONDecodeError:
                    return None
            return None
    return None

In [4]:
def print_json(response):
    json_format = extract_json_from_response(response)
    if json_format is not None:
        pprint.pprint(json_format)
    else:
        print("No valid JSON found in response:", response[:200] + "..." if len(response) > 200 else response)
    return None

In [5]:
url = "http://localhost:11434/api/generate"
headers = {
    "Content-Type": "application/json",
}

def llm(model, prompt):
    data_llm = {
        "model": model,
        "prompt": str(prompt),
        "stream": False,
        "options": {
            "temperature": 0.0
        }

    }

    response = requests.post(url, headers=headers, data=json.dumps(data_llm))
    if response.status_code == 200:
        response_data = response.json()
        text = response_data.get('response', '').strip()
        return text

In [6]:
prompt = "Do you know OWASP Top Ten 2021 categories?"
response = llm("mistral:7b", prompt_formatting("You are a helpful assistant.", prompt))
print(response)

Yes, I do. The OWASP Top Ten is a regularly updated list of the most critical web application security risks to be aware of. As of 2021, the categories are:

1. Injection
2. Broken Authentication
3. Sensitive Data Exposure
4. XML External Entities (XXE)
5. Broken Access Control
6. Security Misconfiguration
7. Cross-Site Scripting (XSS)
8. Insecure Deserialization
9. Using Components with Known Vulnerabilities
10. Insufficient Logging & Monitoring

Each category includes detailed descriptions and recommendations for mitigation. It's essential to understand these risks when developing or maintaining web applications.


Observações relevantes:

Categorias que se mesclam de acordo com a doc do OWASP:


### Examples

#### Broken Access Control Example (A01)

In [24]:
broken_access_control_example = {
  "pr": {
    "title": "Add client routes",
    "description": "See https://code.djangoproject.com/ticket/23381#ticket\n"
  },

  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "Thanks for the contribution. I noticed that the new POST/PUT/DELETE endpoints don't enforce authentication or authorization. This is a classic missing access control issue: an unauthenticated user could create, modify, or delete clients. Please add requireAuth and enforce a clients:write permission (or equivalent role)."
      ]
    }, 
    {
      "scope": "FILE:server/middleware/auth.ts",
      "discussion": [
        "I don't see CSRF protection applied to POST/PUT/DELETE when using session cookies. If your deployment uses cookies, please add csrfProtection() to these routes. If you exclusively use bearer tokens, add a comment linking to the security doc stating why CSRF isn't necessary.",
        "Added conditional CSRF middleware for session mode and a comment in code + docs clarifying the token-based scenario."
      ]
    }
  ]
}

#### Cryptographic Failures Example (A02)

In [25]:
cryptographic_failures_example = {
  "pr": {
    "title": "Add payment webhooks",
    "description": "Implements inbound payment webhooks, verifies JWT signatures, and encrypts stored payloads.\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "During review I spotted deprecated PKCS#1 v1.5 padding in JWT signing (RS256) and RSA encryption used by the new webhooks. Please migrate to RSA-PSS (PS256) and RSA-OAEP with SHA-256, and restrict verification to a safe allowlist.",
        "Updated: JWT now uses PS256 with a strict verify allowlist; encryption switched to OAEP-SHA256. Added brief migration notes and tests."
      ]
    }
  ]
}

#### Injection Example (A03)

In [26]:
injection_example = {
  "pr": {
    "title": "Add customer analytics export feature",
    "description": "Introduces an export endpoint and a CLI to generate customer analytics reports.\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "Security issue: hostile input is concatenated into SQL and shell commands. Please replace dynamic string-built SQL with parameterized queries/whitelisted identifiers, and switch shell execution to spawn/execFile with validated args (no shell interpolation). Add tests for injection attempts.",
        "Fixed: SQL now uses placeholders with a whitelist for sort fields; CLI moved to spawn with args and strict path/flag validation. Added negative tests for SQL/command injection."
      ]
    },
    {
      "scope": "FILE:server/db/reports.ts",
      "discussion": [
        "The query concatenates req.query.status, req.query.since, and req.query.sort into the SQL string. This is vulnerable to injection. Use placeholders ($1, $2, …) for values and a strict allowlist for sortable columns/directions instead of interpolating them.",
        "Updated: values bound via parameters; sort field/direction validated against an allowlist before constructing ORDER BY. No user data is concatenated into SQL."
      ]
    }
  ]
}

#### Insecure Design (A04) -> Alterado

In [27]:
insegure_design_example = {
  "pr": {
    "title": "Add account recovery endpoints and UI",
    "description": "Introduces self-service account recovery and updates the settings page.\n"
  },
  "threads": [
    {
      "scope": "FILE:server/routes/recovery.ts",
      "discussion": [
        "These endpoints accept and compare plaintext answers (e.g., POST /recovery/verify-questions). Remove them. Add endpoints for issuing and verifying email OTPs and for WebAuthn assertion verification.",
        "Done: removed question endpoints; added /recovery/otp/initiate and /recovery/otp/verify plus WebAuthn assertion verify. Added throttling and audit logs."
      ]
    },
    {
      "scope": "FILE:ui/pages/recovery.tsx",
      "discussion": [
        "The form renders security questions and posts user-provided answers. Replace with an email code entry screen and a WebAuthn fallback prompt. Avoid storing or displaying any Q&A content.",
        "Updated UI: email code input and WebAuthn prompt; all Q&A UI removed."
      ]
    }
  ]
}

In [28]:
insegure_design_example_new = {
  "pr": {
    "title": "Add premium subscription tiers",
    "description": "Introduces logic to handle 'Basic' and 'Premium' subscription levels and enables exclusive features for 'Premium' users."
  },
  "threads": [
    {
      "scope": "FILE:server/controllers/features.ts",
      "discussion": [
        "Major design issue: The system determines a user's access to 'Premium' features (like custom reports) by **checking the subscription level exclusively on the client-side** (browser) before making a *server* request. The server-side check only validates that the *request* is authenticated, not *authorized* for the feature. An attacker could easily modify the client-side code or API call to bypass this check.",
        "This logic flaw is an **Insecure Design**. We need to implement a **server-side authorization check** (`if (user.subscription === 'premium')`) before processing any 'Premium' feature logic. Please add a robust server-side check and update the documentation to reflect this required authorization boundary."
      ]
    },
    {
      "scope": "FILE:ui/components/CustomReports.tsx",
      "discussion": [
        "The button to generate custom reports is conditionally rendered based on a client-side flag: `if (currentUser.is_premium) { /* render button */ }`. This is UI-level access control, which is easily bypassed. This is *not* the security issue, but a symptom of the design flaw. The security fix must be on the server."
      ]
    }
  ]
}

#### Security Misconfiguration (A05)

In [29]:
security_misconfiguration_example = {
  "pr": {
    "title": "Deploy customer portal on Tomcat",
    "description": "Adds Dockerfile and Tomcat config to deploy the customer portal WAR with health checks.\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "Prod image still includes Tomcat sample apps (examples, docs, manager, host-manager). These are known-vulnerable and must not be deployed to production. Please remove them and block those context paths.",
        "Fixed: pruned all default webapps, disabled manager/autoDeploy, and added checks ensuring /examples, /docs, /manager, and /host-manager return 404/403."
      ]
    },
    {
      "scope": "FILE:ops/tomcat/server.xml",
      "discussion": [
        "autoDeploy and deployOnStartup expose dropped sample apps if present; also the manager context is enabled. Disable autoDeploy/deployOnStartup and ensure manager/host-manager contexts are not defined.",
        "Set autoDeploy=\"false\" and deployOnStartup=\"false\", removed manager/host-manager contexts, and added a rewrite rule to deny requests to sample paths."
      ]
    }
  ]
}

#### Vulnerable and Outdated Components (A06)

In [30]:
vulnerable_components_example = {
  "pr": {
    "title": "Add ticket attachments upload (Struts 2 module)",
    "description": "Implements a file upload action for support tickets and wires Struts filters.\n"
  },
  "threads": [
    {
      "scope": "PR",
      "discussion": [
        "Security: the change introduces Struts 2 with versions/configs vulnerable to RCE (e.g., CVE-2017-5638, CVE-2018-11776). Please upgrade to Struts 2.5.33+ (or latest), update commons-fileupload to a patched version, disable Dynamic Method Invocation and wildcard actions, and ensure devMode is off.",
        "Updated: bumped Struts to 2.5.33 and commons-fileupload to 1.5, disabled DMI and wildcards, enforced safe mapper settings, and turned off devMode. Added basic tests."
      ]
    },
    {
      "scope": "FILE:pom.xml",
      "discussion": [
        "struts2-core is pinned to 2.3.31 and pulls an old commons-fileupload transitively. Please set struts2-core to >= 2.5.33 and explicitly depend on commons-fileupload 1.5. Consider excluding older transitive versions.",
        "Done: struts2-core 2.5.33, explicit commons-fileupload 1.5, and exclusions added to prevent older artifacts."
      ]
    },
    {
      "scope": "FILE:WEB-INF/struts.xml",
      "discussion": [
        "The config enables wildcard actions and dynamic method resolution, which can be chained into OGNL/RCE issues. Remove wildcard mappings and add: struts.enable.DynamicMethodInvocation=false, struts.mapper.alwaysSelectFullNamespace=true, and devMode=false. Restrict allowedMethods on actions.",
        "Applied: removed wildcards, added the constants, set allowedMethods per action, and verified devMode is false for production."
      ]
    }
  ]
}

#### Identification and Authentication Failures (A07)


In [31]:
identification_and_authentication_failures_example = {
  "pr": {
    "title": "Add tenant setup wizard",
    "description": "Implements initial admin creation and onboarding flow for new tenants.\n"
  },
  "threads": [
    {
      "scope": "FILE:infrastructure/sql/init.sql",
      "discussion": [
        "This seeds a default admin with password 'admin'. Do not create credentials in production images. Remove this and create no user by default.",
        "Done: dropped admin seeding; migrations now create tenant metadata only. Admin is created via setup link without a default password."
      ]
    },
    {
      "scope": "FILE:server/auth/passwordPolicy.ts",
      "discussion": [
        "Current validator permits 'Password1' and similar. Enforce: length >= 12, deny common/well-known passwords, forbid username/email substrings, and require character variety or entropy threshold. Apply in createUser and resetPassword.",
        "Implemented policy and wired into create/reset paths; added tests that reject 'Password1', 'adminadmin', and known-common passwords."
      ]
    }
  ]
}

#### Software and Data Integrity Failures (A08)


In [ ]:
software_and_data_integrity_failures_example = {
  "pr": {
    "title": "Add staging deployment pipeline",
    "description": "Sets up a GitHub Actions workflow to build Docker images and deploy to the staging cluster.\n"
  },
  "threads": [
    {
      "scope": "FILE:.github/workflows/deploy.yml",
      "discussion": [
        "Workflow uses floating versions (e.g., actions/checkout@v4) and downloads artifacts without integrity checks. Pin to commit SHAs and verify artifact checksums before deployment.",
        "Changed to commit SHA pins for all Actions and added a checksum verification step for the build artifact."
      ]
    }
  ]
}

#### Security Logging and Monitoring Failures (A09)


In [32]:
security_logging_and_monitoring_failures_example = {
  "pr": {
    "title": "Add wire transfer API",
    "description": "Implements endpoints to initiate and approve high-value transfers; updates login handler.\n"
  },
  "threads": [
    {
      "scope": "FILE:server/payments/transfers.ts",
      "discussion": [
        "High-value transfer routes don't log auditable events. Emit events for creation, approval, cancellation, and failed attempts with actorId, transferId, amount/currency, idempotencyKey, and result/status code.",
        "Added audit events for all state changes with required metadata; included tests for success and failure paths."
      ]
    }
  ]
}

#### Server-Side Request Forgery (A10)


In [33]:
server_side_request_forgery_example = {
  "pr": {
    "title": "Add link preview unfurling",
    "description": "Implements an endpoint that fetches a URL and returns basic metadata for chat/link previews.\n"
  },
  "threads": [
    {
      "scope": "FILE:server/utils/url_validation.ts",
      "discussion": [
        "Please implement an allowlist-based validator: only http/https, ports 80/443, resolve host to IP and require public IP, block RFC1918/localhost/link-local/ULA ranges, and re-check after every redirect to prevent DNS rebinding.",
        "Implemented validateRemoteUrl() with isPublicIp() and redirect revalidation; added unit tests for private ranges and rebinding scenarios."
      ]
    }
  ]
}

In [34]:
def call_owasp_examples(system, model="mistral:7b"):
    print("=== Broken Access Control (A01)")
    response = llm(model, prompt_formatting(system, json.dumps(broken_access_control_example)))
    _ = print_json(response)

    print("\n=== Cryptographic Failures (A02)")
    response = llm(model, prompt_formatting(system, json.dumps(cryptographic_failures_example)))
    _ = print_json(response)

    print("\n=== Injection (A03)")
    response = llm(model, prompt_formatting(system, json.dumps(injection_example)))
    _ = print_json(response)

    print("\n=== Insecure Design (A04)")
    response = llm(model, prompt_formatting(system, json.dumps(insegure_design_example)))
    _ = print_json(response)

    print("\n=== Security Misconfiguration (A05)")
    response = llm(model, prompt_formatting(system, json.dumps(security_misconfiguration_example)))
    _ = print_json(response)

    print("\n=== Vulnerable and Outdated Components (A06)")
    response = llm(model, prompt_formatting(system, json.dumps(vulnerable_components_example)))
    _ = print_json(response)

    print("\n=== Identification and Authentication Failures (A07)")
    response = llm(model, prompt_formatting(system, json.dumps(identification_and_authentication_failures_example)))
    _ = print_json(response)

    print("\n=== Software and Data Integrity Failures (A08)")
    response = llm(model, prompt_formatting(system, json.dumps(software_and_data_integrity_failures_example)))
    _ = print_json(response)

    print("\n=== Security Logging and Monitoring Failures (A09)")
    response = llm(model, prompt_formatting(system, json.dumps(security_logging_and_monitoring_failures_example)))
    _ = print_json(response)

    print("\n=== Server-Side Request Forgery (A10)")
    response = llm(model, prompt_formatting(system, json.dumps(server_side_request_forgery_example)))
    _ = print_json(response)
    
    # Explicitly return None to prevent any automatic display
    return None

### Zero-shot Summarization Prompt (without quote OWASP)

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions of vulnerabilities, exploits, secrets, auth/authorization, cryptography, input validation, privacy, data exposure, dependency risk, or hardening. Provide a list of the summarize findings in json format with the key "summarization". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions of vulnerabilities, exploits, secrets, auth/authorization, cryptography, input validation, privacy, data exposure, dependency risk, or hardening. Provide a list of the summarized findings in json format with the key "summarization". Do not add any preambles."""

In [20]:
call_owasp_examples(system)

=== Broken Access Control (A01)


KeyboardInterrupt: 

### Zero-shot Summarization Prompt (quoting OWASP)

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021. Provide a list of the summarize findings in json format with the key "summarization". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021. Provide a list of the summarize findings in json format with the key "summarization". Do not add any preambles."""

In [ ]:
call_owasp_examples(system)

=== Broken Access Control (A01)
{'summarization': ['Missing Access Control Issue (OWASP A6: Broken '
                   "Authentication): The new POST/PUT/DELETE endpoints don't "
                   'enforce authentication or authorization. An '
                   'unauthenticated user could create, modify, or delete '
                   'clients.',
                   'CSRF Protection Issue (OWASP AXS-01: Cross-Site Request '
                   'Forgery): CSRF protection is not applied to '
                   'POST/PUT/DELETE when using session cookies. If your '
                   'deployment uses cookies, please add csrfProtection() to '
                   'these routes.']}

=== Cryptographic Failures (A02)
{'summarization': ['The PR introduces inbound payment webhooks, but it was '
                   'found that deprecated PKCS#1 v1.5 padding is being used in '
                   'JWT signing (RS256) and RSA encryption. The recommended '
                   'action is to migrate to R

### Zero-shot Classification Prompt (quoting OWASP)

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles."""

In [ ]:
call_owasp_examples(system)

=== Broken Access Control (A01)
{'category': 'A1:2021 - Injection', 'findings': []}

=== Cryptographic Failures (A02)
{'category': 'Injection', 'findings': []}

=== Injection (A03)
{'category': 'A1 - Injection',
 'findings': ['Security issue: hostile input is concatenated into SQL and '
              'shell commands in the server/db/reports.ts file. This was fixed '
              'by replacing dynamic string-built SQL with parameterized '
              'queries/whitelisted identifiers, and switching shell execution '
              'to spawn/execFile with validated args (no shell interpolation).',
              'The query in server/db/reports.ts concatenates user input into '
              'the SQL string, making it vulnerable to injection. This was '
              'updated by using placeholders for values and a strict allowlist '
              'for sortable columns/directions instead of interpolating them.']}

=== Insecure Design (A04)
{'category': 'A1 - Injection',
 'findings': ['The 

### Zero-shot Summarization Prompt (quoting OWASP directly, without long explanation)

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. Provide a list of the summarized findings in json format with the key "summarization". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. Provide a list of the summarized findings in json format with the key "summarization". Do not add any preambles."""

In [ ]:
call_owasp_examples(system)

=== Broken Access Control (A01)
{'summarization': ['Missing Access Control: The new POST/PUT/DELETE endpoints '
                   'do not enforce authentication or authorization, allowing '
                   'unauthenticated users to create, modify, or delete '
                   'clients. (PR)',
                   'Cryptographic Failures/Sensitive Data Exposure: No CSRF '
                   'protection is applied to POST/PUT/DELETE when using '
                   'session cookies. If your deployment uses cookies, please '
                   'add csrfProtection() to these routes. If you exclusively '
                   'use bearer tokens, add a comment linking to the security '
                   "doc stating why CSRF isn't necessary. "
                   '(FILE:server/middleware/auth.ts)']}

=== Cryptographic Failures (A02)
{'summarization': ['The pull request introduces inbound payment webhooks and '
                   'addresses issues related to Cryptographic '
                  

### Zero-shot Categorization Prompt (quoting OWASP directly, without long explanation)

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles."""

In [ ]:
call_owasp_examples(system)

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'findings': ["The new POST/PUT/DELETE endpoints don't enforce authentication "
              'or authorization. An unauthenticated user could create, modify, '
              'or delete clients.']}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'findings': ['Deprecated PKCS#1 v1.5 padding in JWT signing (RS256) and RSA '
              'encryption used by the new webhooks.',
              'RSA-PSS (PS256) and RSA-OAEP with SHA-256 were suggested as '
              'replacements for the deprecated methods.']}

=== Injection (A03)
{'category': 'Injection',
 'findings': ["The query in the file 'server/db/reports.ts' was concatenating "
              'user input into the SQL string, making it vulnerable to '
              'injection.',
              'The issue has been fixed by using placeholders for values and '
              'validating sortable columns/directions befor

### Zero-shot Categorization Prompt (quoting OWASP directly, without long explanation) with some improviments

#### Adding input description: 

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a list of the summarized findings in json format with the key "category" and the key "findings". Do not add any preambles."""
call_owasp_examples(system)

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'findings': ["The new POST/PUT/DELETE endpoints don't enforce authentication "
              'or authorization. An unauthenticated user could create, modify, '
              'or delete clients. Please add requireAuth and enforce a '
              'clients:write permission (or equivalent role).']}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'findings': ["In the pull request 'Add payment webhooks', there was a use of "
              'deprecated PKCS#1 v1.5 padding in JWT signing (RS256) and RSA '
              'encryption. The issue has been resolved by migrating to RSA-PSS '
              '(PS256) and RSA-OAEP with SHA-256, and restricting verification '
              'to a safe allowlist.']}

=== Injection (A03)
{'category': 'Injection',
 'findings': ['SQL injection vulnerability in the query construction of '
              'server/db/reports.ts, now fixed by usi

#### Adding output improved description: 

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "category", which is the category cited with greatest correlation with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "category", which is the category cited with greatest correlation with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles."""

In [ ]:
call_owasp_examples(system)

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'finding': 'The new POST/PUT/DELETE endpoints do not enforce authentication '
            'or authorization, allowing unauthenticated users to create, '
            'modify, or delete clients.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'finding': 'Deprecated PKCS#1 v1.5 padding in JWT signing (RS256) and RSA '
            'encryption used by the new webhooks have been replaced with '
            'RSA-PSS (PS256) and RSA-OAEP with SHA-256, and verification has '
            'been restricted to a safe allowlist.'}

=== Injection (A03)
{'category': 'Injection',
 'finding': 'The initial pull request introduced SQL and command injection '
            'vulnerabilities by concatenating unvalidated user input into SQL '
            'strings and shell commands. This was addressed by replacing '
            'dynamic string-built SQL with parameterized queries/whiteliste

In [ ]:
print("\n=== New Insecure Design (A04)")
response = llm("mistral:7b", prompt_formatting(system, json.dumps(insegure_design_example_new)))
_ = print_json(response)


=== New Insecure Design (A04)


{'category': 'Insecure Design',
 'finding': "The system determines a user's access to 'Premium' features "
            'exclusively on the client-side before making a server request. '
            "Implement a server-side authorization check for 'Premium' feature "
            'logic.'}


#### Adding output allowing multiple categories: 

You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "categories", which is a list of relevant categories correlated with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles.

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "categories", which is a list of relevant categories correlated with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles."""
call_owasp_examples(system)

=== Broken Access Control (A01)
{'categories': ['Broken Access Control', 'Insecure Design'],
 'finding': 'The pull request introduces new endpoints without enforcing '
            'authentication or authorization, allowing unauthenticated users '
            'to create, modify, or delete clients.'}

=== Cryptographic Failures (A02)
{'categories': ['Cryptographic Failures/Sensitive Data Exposure'],
 'finding': 'Initial implementation used deprecated RSA padding (PKCS#1 v1.5) '
            'in JWT signing and RSA encryption for webhooks. This has been '
            'updated to use RSA-PSS (PS256) and RSA-OAEP with SHA-256, and '
            'verification has been restricted to a safe allowlist.'}

=== Injection (A03)
{'categories': ['Injection'],
 'finding': 'SQL injection vulnerability found in the query construction of '
            'server/db/reports.ts. The issue has been resolved by using '
            'placeholders and validating sortable columns/directions.'}

=== Insecure Design 

### Zero-shot Categorization Prompt (quoting OWASP directly, without long explanation) with other LLMs

#### With small LLMs


In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "category", which is the category cited with greatest correlation with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles."""
call_owasp_examples(system, model="qwen3:1.7b")

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'finding': 'Missing authentication and authorization checks in '
            'POST/PUT/DELETE endpoints for client routes, allowing '
            'unauthenticated users to modify/delete clients.'}

=== Cryptographic Failures (A02)
{'findings': [{'category': 'Cryptographic Failures/Sensitive Data Exposure',
               'finding': 'Deprecated PKCS#1 v1.5 padding in JWT signing '
                          '(RS256) is used, which is a cryptographic failure. '
                          'Migration to RSA-PSS and OAEP with SHA-256 is '
                          'recommended to ensure secure cryptographic '
                          'practices.'}]}

=== Injection (A03)
{'category': 'Injection',
 'finding': 'SQL injection vulnerability in the export endpoint, where user '
            'input is directly concatenated into SQL queries. Fixed by using '
            'parameterized queries and validating sort fields.'}

=== Ins

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "category", which is the category cited with greatest correlation with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles."""
call_owasp_examples(system, model="deepseek-r1:1.5b")

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'finding': 'Missing authentication middleware prevents users from creating, '
            'modifying, or deleting clients without proper roles.'}

=== Cryptographic Failures (A02)
{'category': 'Broken Access Control',
 'finding': 'Messages indicate the use of deprecated PKCS#1 v1.5 padding in '
            'JWT signing (RS256), which may lead to unauthorized access if not '
            'properly updated.'}

=== Injection (A03)
{'category': 'Broken Access Control',
 'finding': 'The messages highlight vulnerabilities in query parameter '
            'manipulation and SQL injection risks within the server-side '
            'application.'}

=== Insecure Design (A04)
{'category': 'Account Recovery Endpoints',
 'finding': 'The pull request introduces self-service account recovery '
            'endpoints which are replaced with secure alternatives.'}

=== Security Misconfiguration (A05)
{'category': 'Broken Access Contro

### With other average LLMs

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide the summarized findings in JSON format. Each finding must be an object containing the keys "category", which is the category cited with greatest correlation with the problem, and "finding", which is a concise summary of the finding itself. Do not add any preambles."""
call_owasp_examples(system, model="qwen3:8b")

=== Broken Access Control (A01)


{'category': 'Broken Access Control',
 'finding': 'New endpoints lack authentication/authorization enforcement, '
            'allowing unauthenticated users to create, modify, or delete '
            'clients.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures',
 'finding': 'Deprecated PKCS#1 v1.5 padding in JWT signing (RS256) and RSA '
            'encryption were identified, requiring migration to RSA-PSS '
            '(PS256) and RSA-OAEP with SHA-256, along with strict verification '
            'allowlists.'}

=== Injection (A03)
{'category': 'Injection',
 'finding': 'The PR addresses SQL injection vulnerabilities by replacing '
            'concatenated SQL queries with parameterized queries and '
            'implementing a whitelist for sort fields. It also fixes command '
            'injection risks by using spawn/execFile with validated arguments '
            'and strict path/flag validation.'}

=== Insecure Design (A04)
{'category': 'Sensitive Data

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a list of the summarized findings in json format with each finding containing the key "category", with only one of the mentioned categories, and the key "findings", with the summarized finding. Do not add any preambles."""
call_owasp_examples(system, model="llama3:latest")

=== Broken Access Control (A01)


{'category': 'Broken Access Control',
 'findings': 'The new POST/PUT/DELETE endpoints do not enforce authentication '
             'or authorization, allowing unauthenticated users to create, '
             'modify, or delete clients.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'findings': 'The pull request contains deprecated PKCS#1 v1.5 padding in JWT '
             'signing (RS256) and RSA encryption used by the new webhooks, '
             'which should be migrated to RSA-PSS (PS256) and RSA-OAEP with '
             'SHA-256.'}

=== Injection (A03)
{'category': 'Injection',
 'findings': 'The pull request contains a security issue where hostile input '
             'is concatenated into SQL and shell commands, which can lead to '
             'injection attacks. The issue has been fixed by using '
             'parameterized queries with whitelisted identifiers for SQL and '
             'validated arguments for shell execution

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a list of the summarized findings in json format with each finding containing the key "category", with only one of the mentioned categories, and the key "findings", with the summarized finding. Do not add any preambles."""
call_owasp_examples(system, model="gemma3:latest")

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'findings': 'Unauthenticated users could create, modify, or delete clients '
             'due to missing authentication and authorization on new '
             'POST/PUT/DELETE endpoints.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'findings': 'Deprecated PKCS#1 v1.5 padding in JWT signing and RSA encryption '
             'used by the new webhooks. Migration to RSA-PSS (PS256) and '
             'RSA-OAEP with SHA-256 is required.'}

=== Injection (A03)
{'category': 'Injection',
 'findings': 'The PR description and initial thread identified a vulnerability '
             'where hostile input is concatenated into SQL and shell commands. '
             'The fix involved using parameterized queries and strict '
             'path/flag validation.'}

=== Insecure Design (A04)
{'category': 'Insecure Design',
 'findings': 'The PR discusses accepting and comparing p

### With bigger LLMs

In [ ]:
system = """You are an AppSec analyst focused on identifying security-relevant signals in pull request discussions. Scan the PR metadata and discussion for mentions that align with problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures/Sensitive Data Exposure, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has an "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a list of the summarized findings in json format with each finding containing the key "category", with only one of the mentioned categories, and the key "findings", with the summarized finding. Do not add any preambles."""
call_owasp_examples(system, model="gemma3:12b")

=== Broken Access Control (A01)


{'category': 'Broken Access Control',
 'findings': 'New POST/PUT/DELETE endpoints for clients lack authentication '
             'and authorization, potentially allowing unauthenticated users to '
             'create, modify, or delete clients. Requires adding `requireAuth` '
             'and enforcing appropriate permissions.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures/Sensitive Data Exposure',
 'findings': 'Identified use of deprecated PKCS#1 v1.5 padding in JWT signing '
             '(RS256) and RSA encryption. Remediation involved migrating to '
             'PS256 and RSA-OAEP with SHA-256.'}

=== Injection (A03)
{'category': 'Injection',
 'findings': 'Initial code concatenated user-supplied data (req.query.status, '
             'req.query.since, req.query.sort) directly into SQL queries, '
             'creating a SQL injection vulnerability. The fix implemented '
             'parameterized queries with a whitelist for sort fields and '
         

You are an Application Security analyst focused on identifying security-relevant issues in pull request discussions. Scan the PR metadata and discussion for mentions that align with the following problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has a "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a summary of the identified issue in JSON format. Each issue must be an object containing the keys "category", which is the category cited with greatest relation with the problem, and "issue", which is a concise summary of the issue itself. Do not add any preambles.

You are an Application Security analyst focused on identifying security-relevant issues in pull request discussions. Scan the PR metadata and discussion for mentions that align with the following problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has a "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a summary of the identified issues in JSON format. Each issue must be an object containing the keys "category", which is the category cited with greatest relation with the problem, and "issue", which is a concise summary of the issue itself. If no match is found for a given category, do not include it in the output. Do not add any preambles.

In [39]:
system = """You are an Application Security analyst focused on identifying security-relevant issues in pull request discussions. Scan the PR metadata and discussion for mentions that align with the following problems mentioned by OWASP Top Ten 2021 categories: Broken Access Control, Cryptographic Failures, Injection, Insecure Design, Security Misconfiguration, Vulnerable and Outdated Components, Identification and Authentication, Software and Data Integrity Failures, Security Logging and Monitoring Failures, Server-Side Request Forgery. As input, you will receive a JSON object that contains a key "pr" with the pull request "title" and "description", and a key "threads" which is an array of discussion threads; each thread has a "scope" indicating the location context and "discussion" as an array of message strings in chronological order. Provide a summary of the identified issue in JSON format. Each issue must be an object containing the keys "category", which is the category cited with greatest relation with the problem, and "issue", which is a concise summary of the issue itself. Do not add any preambles."""
call_owasp_examples(system, model="mistral:7b")

=== Broken Access Control (A01)
{'category': 'Broken Access Control',
 'issue': 'The pull request introduces new endpoints without enforcing '
          'authentication or authorization, allowing unauthenticated users to '
          'create, modify, or delete clients.'}

=== Cryptographic Failures (A02)
{'category': 'Cryptographic Failures',
 'issue': 'The pull request introduces deprecated RSA encryption (RSA v1.5 '
          'padding) and RS256 JWT signing. The recommended practice is to use '
          'RSA-PSS (PS256) for signing and RSA-OAEP with SHA-256 for '
          'encryption, along with a strict verify allowlist.'}

=== Injection (A03)
{'category': 'Injection',
 'issue': 'The application is vulnerable to SQL injection due to dynamic '
          'string-building in the SQL queries. The issue has been addressed by '
          'replacing dynamic string-building with parameterized queries and '
          'whitelisting identifiers.'}

=== Insecure Design (A04)
{'category': 'Brok